In [9]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import feature_spaces
import pandas as pd
from typing import List
from os.path import join
import numpy as np
import encoding_utils, feature_spaces
import pickle as pkl
from collections import defaultdict
from feature_spaces import *
NUM_VOXELS = 250

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
train_stories, test_stories, allstories = encoding_utils.get_allstories([1, 2, 3, 4, 5])
wordseqs = feature_spaces.get_story_wordseqs(test_stories)

In [12]:
def get_words_for_story(wordseq, max_running_words=5) -> List[str]:
    running_words = []
    
    words = np.array(wordseq.data)
    TRIM = 5
    tr_times = wordseq.tr_times[5+TRIM: -TRIM]
    for i, tr_time in enumerate(tr_times):
        valid_times = wordseq.data_times <= tr_time
        # print(valid_times)
        running_words.append(' '.join(words[valid_times][-max_running_words:]))
    #     print(tr_time, running_words)
    return running_words

texts_list_test = []
for story_name in tqdm(test_stories):
    wordseq = wordseqs[story_name]
    texts_list_test.append(get_words_for_story(wordseq))
texts_test = sum(texts_list_test, [])

100%|██████████| 1/1 [00:00<00:00, 324.31it/s]


In [13]:
subj = 'UTS03'

# select top_idxs
save_dir = '/home/chansingh/mntv1/deep-fMRI/results/encoding/bert-10__ndel=4/UTS03'
corrs_val = np.load(join(save_dir, 'corrs.npz'))['arr_0']
top_idxs = np.argsort(corrs_val)[::-1][:NUM_VOXELS]
print(corrs_val[top_idxs][:5])

# load responses (n_time_points x n_voxels)
resp_test = encoding_utils.get_response(test_stories, subj)

[0.76296981 0.74258237 0.72107898 0.71616266 0.71508206]


In [14]:
# get the top ngrams for each voxel
NUM_NGRAMS = 30
top_ngrams = {}
for i, voxel in enumerate(tqdm(top_idxs)):
    top_resp_idxs = np.argsort(resp_test[:, i])[::-1]
    top_ngrams[voxel] = [texts_test[idx][:NUM_NGRAMS] for idx in top_resp_idxs]
top_ngrams = pd.DataFrame.from_dict(top_ngrams)
top_ngrams.columns = [f'voxel_top_{i}' for i in range(NUM_VOXELS)]

100%|██████████| 250/250 [00:00<00:00, 18276.79it/s]


In [15]:
top_ngrams.head(n=5)

,voxel_top_0,voxel_top_1,voxel_top_2,voxel_top_3,voxel_top_4,voxel_top_5,voxel_top_6,voxel_top_7,voxel_top_8,voxel_top_9,...,voxel_top_240,voxel_top_241,voxel_top_242,voxel_top_243,voxel_top_244,voxel_top_245,voxel_top_246,voxel_top_247,voxel_top_248,voxel_top_249
0,i didn't have my cigarettes,a drag of his cigarette,me over and he sees,little crack of light and,and the person comes closer,reminisce about our thirty sec,not ever see her again,washing dogs she had horses,i'll give you a ride,out on this suburban street,...,door and i ran i,was crying i had no,been laundered at least once,i didn't want any part,just then the girl reaches,newish to the neighborhood thi,paper seven cigarettes for me,and we both need this,kind of does the opposite,have my driver's license yet
1,happy to see me and,fun and you know let's,fun and you know let's,and she pulls out a,yeah ok and i walk,i'm surprised when i see,her cutoffs in the front,for anything like we would,there is one match inside,of that cigarette and we,...,are getting wider and wider,you and she says never,and she still understands how,live with someone who has,and i say you got,shoes on i was crying,oh man that was close,halfway there i'm a waitress,live with someone who has,but yes carl's efficiency apar
2,going home now to my,tell people at home this,i didn't want any part,not gonna be friends i,dollar stores and emergecenter,i cross and i walk,and she pulls out a,or drunk or lonely or,little scary and i follow,i'm surprised when i see,...,fight about and i say,be but i know it's,up and there is one,checking and checking and it's,i didn't want any part,but i was ok because,we finish our cigarettes she,have a one bedroom apartment,smoked all seven cigarettes on,i didn't have my cigarettes
3,and the person comes closer,checking and checking and it's,oh hey i'm not a,sweet that'll be we are,checking and checking and it's,not gonna be friends i,temper a very very bad,door and i ran i,{cg} and we share some,i'm back and he says,...,we start walking and uh,and she still understands how,door and i ran i,that was great to meet,halfway there i'm a waitress,i didn't want any part,uh she leads me to,that you learn this time,not gonna be friends i,was crying i had no
4,there and i'm going home,and that was fun and,of that cigarette and we,he keeps me going she,on that has fifty million,lucky i saw you yeah,i'm surprised when i see,on that has fifty million,yeah ok and i walk,he says who the fuck,...,was crying i had no,i can't ever tell people,shoes on i was crying,stuff about what our lives,place and i think no,to trade stories about our,horrible misery going on there,i'll just be real nice,because i had my cigarettes,think that was gonna happen


In [16]:
top_ngrams.to_pickle('top_ngrams.pkl')

In [17]:
!cp top_ngrams.pkl ../../mprompt/mprompt/modules/fmri/